In [2]:
import sys, os
import numpy as np

sys.path.insert(0, "/Users/Shared/auto/07p/python")
#sys.path.insert(0, "/Users/ielyaascloete/auto/07p/python")
from auto import *


In [3]:
cl

<function AUTOCommands.cl>

In [4]:
import sys
print(sys.version)

2.7.14 |Anaconda custom (64-bit)| (default, Dec  7 2017, 11:07:58) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]


In [5]:
Eq = run(e='base_model_hep',c='base_model_hep.eq')

Starting base_model_hep ...

  BR    PT  TY  LAB     V_plc        L2-NORM       PAR(5)        PAR(6)        PAR(7)        PAR(8)        PAR(40)       PAR(41)    
   1     1  EP    1   0.00000E+00   4.15984E+00   8.62898E-02   4.13719E+00   4.24888E-01   0.00000E+00   8.62898E-02   4.13719E+00
   1   200        2   3.09170E-02   4.02135E+00   8.77791E-02   3.99959E+00   4.08254E-01   1.34549E-02   8.77791E-02   3.99959E+00
   1   400        3   4.73531E-02   3.82247E+00   9.02629E-02   3.80225E+00   3.81591E-01   2.12595E-02   9.02629E-02   3.80225E+00
   1   600        4   5.84989E-02   3.62334E+00   9.32166E-02   3.60492E+00   3.51695E-01   2.71982E-02   9.32166E-02   3.60492E+00
   1   605  HB    5   5.87046E-02   3.61907E+00   9.32856E-02   3.60070E+00   3.51021E-01   2.73155E-02   9.32856E-02   3.60070E+00
   1   800        6   6.70303E-02   3.42503E+00   9.66867E-02   3.40860E+00   3.19125E-01   3.23863E-02   9.66867E-02   3.40860E+00
   1  1000        7   7.40325E-02   3.22622E+0

Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL


In [8]:
PO1 = run(Eq('HB1'),c ='base_model_hep.per',ILP=1,NMX=2000)

Starting base_model_hep ...

  BR    PT  TY  LAB     V_plc        L2-NORM        PERIOD       PAR(5)        PAR(6)        PAR(7)        PAR(8)        PAR(40)    
   5   296  LP   28   5.86111E-02   3.59426E+00   6.55586E+01   1.04672E-01   3.59707E+00   3.62288E-01   3.06096E-02   8.48614E-02
   5   300       29   5.86112E-02   3.59301E+00   6.64942E+01   1.05031E-01   3.59653E+00   3.62548E-01   3.07086E-02   8.46640E-02
   5   324  PD   30   5.86133E-02   3.58563E+00   7.18191E+01   1.07089E-01   3.59334E+00   3.64059E-01   3.12700E-02   8.35883E-02
   5   471  LP   31   5.86443E-02   3.46657E+00   1.30772E+02   1.47906E-01   3.54633E+00   3.88333E-01   3.99924E-02   7.31063E-02
   5   499  LP   32   5.86422E-02   3.42975E+00   1.44003E+02   1.90031E-01   3.53512E+00   3.94975E-01   4.51495E-02   7.04130E-02
   5   500  LP   33   5.86422E-02   3.42870E+00   1.44456E+02   1.94545E-01   3.53505E+00   3.95021E-01   4.55339E-02   7.02987E-02
   5   508  BP   34   5.86437E-02   3.43300E+0

Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG IEEE_DIVIDE_BY_ZERO IEEE_OVERFLOW_FLAG IEEE_UNDERFLOW_FLAG IEEE_DENORMAL


In [9]:
import pandas as pd
import matplotlib.pyplot as plt

In [10]:
def auto_to_csv(branch,name,solution_type):
    """ The goal of this function is to save as a csv file the results of an AUTO continuation 
        and return a panda dataframe of the data"""
    """ Input:
        branch: an AUTO branch solution acquired using run(blablabla)
        name: The name of the output csv file, must be a string
        solution_type: 'EQ' for equilibrium, 'PR' for periodic orbit, 'HB' for Hopf two parameters continuation
            'LPPR' Limit point of periodic orbit
        
        Output:
        panda dataframe of the branch solution"""
    
    
    """Relabelling the branch solution """
    branch = rl(branch)
    """Saving the branch solution"""
    sv(branch,name)
    
    """Acquire the b. file"""
    print("Acquire the b. file")
    name_b = 'b.' + name
    content = None
    with open(name_b, 'r') as f:
        content = f.readlines()
    
    
    print("Read file")
    """Read the file from the beginning of the interesting part (depends on its type)"""
    
    for i in range(len(content)):
    
        if "PT" in content[i]:
            init = i
            break



            
    content_csv = [[el for el in content[init].split(' ') if len(el) > 0 and el != '\n']]
    print(content_csv)
        
    
        
    print("Rename branch")
    """ Rename the branch"""
    content_csv[0][0] = 'branch'
    column_names = content_csv[0]
    
    
    
    print("Split content")
    """Split and read the content"""
    for line in content:
        dummy = line.split(' ')
        dummy = [el for el in dummy if len(el) > 0 and el != '\n']
        if dummy[0] == '0':
            continue

        for el_i, el in enumerate(dummy):
            if el_i < 4:
                dummy[el_i] = int(el)
            else:
                dummy[el_i] = float(el)

        if len(dummy) > 1:
            content_csv.append(dummy)
        
    # transform into panda data frame
        
    df = pd.DataFrame(content_csv, columns=column_names)
    
    # Export to csv
    name_csv = name + '.csv'
    df[1:-1].to_csv(name_csv)

        
    
    
    
    return df[1:-1]

In [11]:
df_eq = auto_to_csv(Eq,'Eq','EQ')

Relabeling done
Saving to b.Eq, s.Eq, and d.Eq ... done
Acquire the b. file
Read file
[['0', 'PT', 'TY', 'LAB', 'V_plc', 'L2-NORM', 'PAR(5)', 'PAR(6)', 'PAR(7)', 'PAR(8)', 'PAR(40)', 'PAR(41)', 'PAR(42)', 'PAR(43)', 'PAR(80)']]
Rename branch
Split content


In [12]:
df_PR = auto_to_csv(PO1,'PO1','PR')

Relabeling done
Saving to b.PO1, s.PO1, and d.PO1 ... done
Acquire the b. file
Read file
[['0', 'PT', 'TY', 'LAB', 'V_plc', 'L2-NORM', 'PERIOD', 'PAR(5)', 'PAR(6)', 'PAR(7)', 'PAR(8)', 'PAR(40)', 'PAR(41)', 'PAR(42)', 'PAR(43)', 'PAR(80)']]
Rename branch
Split content


In [13]:
import plotly.plotly as py
from plotly.graph_objs import *
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

In [14]:
trace_eq = Scatter(x=df_eq['V_plc'],
                  y=df_eq['PAR(5)'],
                  name='Equilibria',
                  mode = 'lines')
trace_per = Scatter(x=df_PR['V_plc'],
                  y=df_PR['PAR(5)'],
                  name='Periodic Orbit',
                   mode='lines')
trace_per_min = Scatter(x=df_PR['V_plc'],
                  y=df_PR['PAR(40)'],
                  name='Periodic Orbit',
                   mode='lines')
data = [trace_eq,trace_per,trace_per_min]
layout = Layout(title="Bifurcation diagram",
               xaxis=dict(title='V_plc'),
               yaxis=dict(title='[Ca]'))
fig = Figure(data=data, layout=layout)
iplot(fig)

In [ ]:
Is_1 = run(e='base_model_hep_dat',c='base_model_hep.per',dat='bm_per3',ISW=1,ISP=2,NPR=10)
Is_2 = run(e='base_model_hep_dat',c='base_model_hep.per',dat='bm_per3', DS='-',ISW=1,SP=['BP0'],ISP=2,NPR=10)
Is = Is_1 + Is_2

In [ ]:
Is_3 = run(e='base_model_hep_dat',c='base_model_hep.per',dat='bm_per45',ISW=1,PAR={1:0.45},ISP=2,EPSL= 1e-12, EPSU = 1e-12,NTST=200)
Is_4 = run(e='base_model_hep_dat',c='base_model_hep.per',dat='bm_per45', DS='-',ISW=1,SP=['BP0'],PAR={1:0.45},ISP=2,EPSL= 1e-12, EPSU = 1e-12,NTST=200)
Is2 = Is_3 + Is_4

In [ ]:
Is_5 = run(e='base_model_hep_dat',c='base_model_hep.per',dat='bm_per5',ISW=1,PAR={1:0.5},ISP=2,EPSL= 1e-12, EPSU = 1e-12,NTST=200)
Is_6 = run(e='base_model_hep_dat',c='base_model_hep.per',dat='bm_per5', DS='-',ISW=1,SP=['BP0'],PAR={1:0.5},ISP=2,EPSL= 1e-12, EPSU = 1e-12,NTST=200)
Is3 = Is_5 + Is_6

In [ ]:
Is_7 = run(e='base_model_hep_dat',c='base_model_hep.per',dat='bm_per51',ISW=1,PAR={1:0.51},ISP=2,EPSL= 1e-12, EPSU = 1e-12,NTST=200)
Is_8 = run(e='base_model_hep_dat',c='base_model_hep.per',dat='bm_per51', DS='-',ISW=1,SP=['BP0'],PAR={1:0.51},ISP=2,EPSL= 1e-12, EPSU = 1e-12,NTST=200)
Is4 = Is_7 + Is_8

In [ ]:
Is_9 = run(e='base_model_hep_dat',c='base_model_hep.per',dat='bm_per52',ISW=1,PAR={1:0.52},ISP=2,EPSL= 1e-12, EPSU = 1e-12,NTST=200)
Is_10 = run(e='base_model_hep_dat',c='base_model_hep.per',dat='bm_per52', DS='-',ISW=1,SP=['BP0'],PAR={1:0.52},ISP=2,EPSL= 1e-12, EPSU = 1e-12,NTST=200)
Is5 = Is_9 + Is_10

In [ ]:
Is_11 = run(e='base_model_hep_dat',c='base_model_hep.per',dat='bm_per53',ISW=1,PAR={1:0.53},ISP=2,EPSL= 1e-12, EPSU = 1e-12,NTST=200)
Is_12 = run(e='base_model_hep_dat',c='base_model_hep.per',dat='bm_per53', DS='-',ISW=1,SP=['BP0'],PAR={1:0.53},ISP=2,EPSL= 1e-12, EPSU = 1e-12,NTST=200)
Is6 = Is_11 + Is_12

In [ ]:
Is_13 = run(e='base_model_hep_dat',c='base_model_hep.per',dat='bm_per525',ISW=1,PAR={1:0.525},ISP=2,EPSL= 1e-12, EPSU = 1e-12,NTST=200)
Is_14 = run(e='base_model_hep_dat',c='base_model_hep.per',dat='bm_per525', DS='-',ISW=1,SP=['BP0'],PAR={1:0.525},ISP=2,EPSL= 1e-12, EPSU = 1e-12,NTST=200)
Is7 = Is_13 + Is_14

In [ ]:
df_Is = auto_to_csv(Is,'Is','IS')

In [ ]:
# df_Is_3 = auto_to_csv(Is_3,'Is_3','IS_3')
# df_Is_4 = auto_to_csv(Is_4,'Is_4','IS_4')
df_Is2 = auto_to_csv(Is2,'Is2','IS2')

In [ ]:
df_Is3 = auto_to_csv(Is3,'Is3','IS3')

In [ ]:
df_Is4 = auto_to_csv(Is4,'Is4','IS4')

In [ ]:
df_Is5 = auto_to_csv(Is5,'Is5','IS5')

In [ ]:
df_Is6 = auto_to_csv(Is6,'Is6','IS6')

In [ ]:
df_Is7 = auto_to_csv(Is7,'Is7','IS7')

In [ ]:
trace_eq = Scatter(x=df_eq['V_plc'],
                  y=df_eq['PAR(5)'],
                  name='Equilibria',
                  mode = 'lines')
trace_per = Scatter(x=df_PR['V_plc'],
                  y=df_PR['PAR(5)'],
                  name='Periodic Orbit',
                   mode='lines')
trace_per_min = Scatter(x=df_PR['V_plc'],
                  y=df_PR['PAR(40)'],
                  name='Periodic Orbit',
                   mode='lines')
tr_is = Scatter(x=df_Is['V_plc'],
                  y=df_Is['PAR(5)'],
                  name='Isola',
                  mode='lines')
tr_is3 = Scatter(x=df_Is2['V_plc'],
                  y=df_Is2['PAR(5)'],
                  name='Isola',
                  mode='lines')
tr_is5 = Scatter(x=df_Is3['V_plc'],
                  y=df_Is3['PAR(5)'],
                  name='Isola',
                  mode='lines')
tr_is7 = Scatter(x=df_Is4['V_plc'],
                  y=df_Is4['PAR(5)'],
                  name='Isola',
                  mode='lines')
tr_is9 = Scatter(x=df_Is5['V_plc'],
                  y=df_Is5['PAR(5)'],
                  name='Isola',
                  mode='lines')
tr_is11 = Scatter(x=df_Is6['V_plc'],
                  y=df_Is6['PAR(5)'],
                  name='Isola',
                  mode='lines')
tr_is13 = Scatter(x=df_Is7['V_plc'],
                  y=df_Is7['PAR(5)'],
                  name='Isola',
                  mode='lines')
data = [trace_eq,trace_per,trace_per_min,tr_is,tr_is3,tr_is5,tr_is7,tr_is9,tr_is11,tr_is13]
layout = Layout(title="Bifurcation diagram",
               xaxis=dict(title='V_plc'),
               yaxis=dict(title='[Ca]'))
fig = Figure(data=data, layout=layout)
iplot(fig)

In [ ]:
trace_eq = Scatter(x=df_eq['V_plc'],
                  y=df_eq['L2-NORM'],
                  name='Equilibria',
                  mode = 'lines')
trace_per = Scatter(x=df_PR['V_plc'],
                  y=df_PR['L2-NORM'],
                  name='Periodic Orbit',
                   mode='lines')
trace_per_min = Scatter(x=df_PR['V_plc'],
                  y=df_PR['L2-NORM'],
                  name='Periodic Orbit',
                   mode='lines')
trace_isola = Scatter(x=df_Is['V_plc'],
                  y=df_Is['L2-NORM'],
                  name='Isola',
                  mode='lines')
trace_isola3 = Scatter(x=df_Is2['V_plc'],
                  y=df_Is2['L2-NORM'],
                  name='Isola',
                  mode='lines')
trace_isola5 = Scatter(x=df_Is3['V_plc'],
                  y=df_Is3['L2-NORM'],
                  name='Isola',
                  mode='lines')
trace_isola7 = Scatter(x=df_Is4['V_plc'],
                  y=df_Is4['L2-NORM'],
                  name='Isola',
                  mode='lines')
tr_is9 = Scatter(x=df_Is5['V_plc'],
                  y=df_Is5['L2-NORM'],
                  name='Isola',
                  mode='lines')
tr_is11 = Scatter(x=df_Is6['V_plc'],
                  y=df_Is6['L2-NORM'],
                  name='Isola',
                  mode='lines')
tr_is13 = Scatter(x=df_Is7['V_plc'],
                  y=df_Is7['L2-NORM'],
                  name='Isola',
                  mode='lines')
data = [trace_eq,trace_per,trace_per_min,trace_isola,trace_isola3,trace_isola5,trace_isola7,tr_is9,tr_is11,tr_is13]
layout = Layout(title="Bifurcation diagram",
               xaxis=dict(title='V_plc'),
               yaxis=dict(title='[Ca]'))
fig = Figure(data=data, layout=layout)
iplot(fig)

In [ ]:
df_eqs = []
df_PRs = []
initial = run(e='hepatocyte_npm',c='hepatocyte_npm.eq',DS='-',RL0 = 0.01, ICP =[2])
for R_act in np.linspace(0.04,0.5,200):
    first_eq = run(initial('EP2'),DS='-',RL0 = -10,RL1 = R_act, ICP =[1])
    Eq = run(first_eq('EP1'),c='hepatocyte_npm.eq',ICP =  [2,1,3,5,6,7,8,9,10,80])
    PO = []
    hb_count = 0
    for i in range(len(Eq())):
        if Eq()[i]['TY'] == 'HB':
            hb_count += 1
            PO = PO + run(Eq('HB'+ str(hb_count)),c ='hepatocyte_npm.per',ICP =  [2,11,1,3,5,6,7,8,9,10,80],ILP = 0,ISP = 0)
#     if Eq('HB2'):
#         PO1 = run(Eq('HB1'),c ='hepatocyte_npm.per',ICP =  [2,11,1,3,5,6,7,8,9,10,80],ILP = 0,ISP = 0)
#         PO1 = PO1 + run(Eq('HB2'),c ='hepatocyte_npm.per',ICP =  [2,11,1,3,5,6,7,8,9,10,80],ILP = 0,ISP = 0)
#     elif Eq('HB1'):
#         PO1 = run(Eq('HB1'),c ='hepatocyte_npm.per',ICP =  [2,11,1,3,5,6,7,8,9,10,80],ILP = 0,ISP = 0)
    df_eqs.append(auto_to_csv(Eq,'Eq_R_act'+str(R_act),'EQ'))
    df_PRs.append(auto_to_csv(PO,'PO_R_act'+str(R_act),'PR'))
    

In [ ]:
len(Eq())


In [ ]:
trace_eq = Scatter(x=df_eqs[8]['ct'],
                  y=df_eqs[8]['PAR(5)'],
                  name='Equilibria',
                  mode = 'lines')
trace_per = Scatter(x=df_PRs[8]['ct'],
                  y=df_PRs[8]['PAR(5)'],
                  name='Periodic Orbit',
                   mode='lines')
trace_per_min = Scatter(x=df_PRs[8]['ct'],
                  y=df_PRs[8]['PAR(8)'],
                  name='Periodic Orbit',
                   mode='lines')
data = [trace_eq,trace_per,trace_per_min]
layout = Layout(title="Bifurcation diagram",
               xaxis=dict(title='Agonist'),
               yaxis=dict(title='[Ca]'))
fig = Figure(data=data, layout=layout)
iplot(fig)

In [ ]:
df_per1[1:-1].to_csv('per1.csv')

In [ ]:
df_per2[1:-1].to_csv('per2.csv')

In [ ]:
df_per2[1:-1]